In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import chi2

# Load data
file_path = r"C:\Users\David1\Downloads\XRF CIUL fixed.xlsx"
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Extract columns
channel = data['channel'].values
dchannel = data['dchannel'].values
E = data['E'].values
dE = data['dE'].values

# Define a linear function
def linear(x, a, b):
    return a * x + b

# Perform the linear fit (weighted by dE)
weights = 1 / dE**2
popt, pcov = curve_fit(linear, channel, E, sigma=dE, absolute_sigma=True)

# Extract fit parameters and errors
a, b = popt
da, db = np.sqrt(np.diag(pcov))

# Compute fitted values and residuals
E_fit = linear(channel, a, b)
residuals = E - E_fit

# Compute chi-squared, reduced chi-squared, and p-value
chi_squared = np.sum((residuals / dE) ** 2)
dof = len(E) - len(popt)  # Degrees of freedom
chi_squared_reduced = chi_squared / dof
p_value = 1 - chi2.cdf(chi_squared, dof)

# Print results
print("Linear Fit Results:")
print(f"Slope (a): {a:.6f} ± {da:.6f}")
print(f"Intercept (b): {b:.6f} ± {db:.6f}")
print(f"Chi-squared: {chi_squared:.2f}")
print(f"Reduced Chi-squared: {chi_squared_reduced:.2f}")
print(f"P-value: {p_value:.3e}")

# Plot the data and the fit
plt.figure(figsize=(10, 6))
plt.errorbar(channel, E, xerr=dchannel, yerr=dE, fmt='o', label='Data', capsize=3)
plt.plot(channel, E_fit, 'r-', label=f'Fit: $E = ({a:.4f}) \\cdot \\mathrm{{channel}} + ({b:.4f})$')
plt.xlabel('Channel')
plt.ylabel('Energy (KeV)')
#plt.title('Linear Fit of Energy vs. Channel')
#plt.legend()
plt.grid(True)
plt.show()

# Plot residuals
plt.figure(figsize=(10, 6))
plt.errorbar(channel, residuals, yerr=dE, fmt='o', capsize=3)
plt.axhline(0, color='r', linestyle='--', label='Zero Residual')
plt.xlabel('Channel')
plt.ylabel('Residuals (E - Fit)')
#plt.title('Residuals of Linear Fit')
#plt.legend()
plt.grid(True)
plt.show()
